In [ ]:
import pickle
import h5py

with open('IsaacLab/SAC/logs/sb3/Isaac-Jetbot-Direct-v0/2025-04-25_16-46-03/replay_buffer.pkl', 'rb') as f:
    data = pickle.load(f)

/tmp/ipykernel_579458/3820449545.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


In [ ]:
replay_buffer = data

In [ ]:
# Create an HDF5 file
with h5py.File('replay_buffer_converted.h5', 'w') as h5file:
    
    # Save observations
    obs_group = h5file.create_group('observations')
    for key, value in replay_buffer.observations.items():
        obs_group.create_dataset(key, data=value, compression="gzip")
    
    # Save next observations
    next_obs_group = h5file.create_group('next_observations')
    for key, value in replay_buffer.next_observations.items():
        next_obs_group.create_dataset(key, data=value, compression="gzip")
    
    # Save actions, rewards, dones
    h5file.create_dataset('actions', data=replay_buffer.actions, compression="gzip")
    h5file.create_dataset('rewards', data=replay_buffer.rewards, compression="gzip")
    h5file.create_dataset('dones', data=replay_buffer.dones, compression="gzip")

print("✅ Done converting to HDF5!")

✅ Done converting to HDF5!


In [ ]:
import d3rlpy

# setup replay CartPole-v0 dataset and environment
dataset, env = d3rlpy.datasets.get_dataset("cartpole-replay")

/home/nitesh/workspace/offline_rl_test/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-26 15:52.32 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(4,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-04-26 15:52.32 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-04-26 15:52.32 [info     ] Action size has been automatically determined. action_size=2


In [5]:
import pickle
import numpy as np
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer, Episode

# Load your sb3 replay buffer
with open('IsaacLab/SAC/logs/sb3/Isaac-Jetbot-Direct-v0/2025-04-25_16-46-03/replay_buffer.pkl', 'rb') as f:
    sb3_buffer = pickle.load(f)

/tmp/ipykernel_599168/3250312434.py:7: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  sb3_buffer = pickle.load(f)


In [ ]:
import gym
from stable_baselines3.common.buffers import ReplayBuffer

from d3rlpy.dataset import MDPDataset


def to_mdp_dataset(replay_buffer: ReplayBuffer) -> MDPDataset:
    pos = replay_buffer.size()
    goal_index = replay_buffer.observations["goal_index"].flatten()
    # poses = replay_buffer.observations["poses"].flatten()
    rgb = replay_buffer.observations["rgb"].flatten()
    
    obs = np.concatenate([goal_index, rgb], axis=0)
    return MDPDataset(observations=obs[:pos, 0],
                      actions=replay_buffer.actions[:pos, 0],
                      rewards=replay_buffer.rewards[:pos, 0],
                      terminals=replay_buffer.dones[:pos, 0],
                      discrete_action=isinstance(replay_buffer.action_space,
                                                 gym.spaces.Discrete))


def to_sb3_predict(d3rlpy_model):
    d3rlpy_model._predict = d3rlpy_model.predict
    # patch predict
    def sb3_predict(observation, state=None, mask=None, deterministic=True):
        if deterministic:
            return d3rlpy_model._predict(observation), None
        return d3rlpy_model.sample_action(observation), None
    d3rlpy_model.predict = sb3_predict
    return d3rlpy_model

In [7]:
mdp_dataset  = to_mdp_dataset(sb3_buffer)

MemoryError: Unable to allocate 146. GiB for an array with shape (19630041600,) and data type float64